In [ ]:
# import sys
# sys.path.append('../')
# import numpy as np
# import scipy
# import tqdm
# import matplotlib.pyplot as plt
# import glob

# import utils.utils
# import utils.geometry

# from core import *


import sys
sys.path.append('../utils')
sys.path.append('../core')
sys.path.append('../')
import numpy as np
import scipy
import tqdm
import utils
import geometry
from core import *

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import glob

# Phantom

In [ ]:
test_phantom = phantom.Phantom(source_path = None,
            voxel_dims = (0.1e-3,0.1e-3,0.1e-3),
            matrix_dims = (400,400,400),
            baseline = (1500, 1000),
            seed = 5678,)

blood = tissue.Tissue(name='blood', c=1578, rho=1060, sigma=11.3, scale=0.00001, label=1)
myocardium = tissue.Tissue(name='myocardium', c=1561.3, rho=1081, sigma=30, scale=0.0001, label=2)
fat = tissue.Tissue(name='fat', c=1450, rho=920, sigma=10, scale=0.0001, label=3)

pos = 0.7

test_phantom.add_tissue_sphere((pos,-pos,0), 8, blood)
test_phantom.add_tissue_sphere((pos,0,0), 16, blood)
test_phantom.add_tissue_sphere((pos,pos,0), 32, blood)

test_phantom.add_tissue_sphere((-pos,-pos,0), 16, fat)
test_phantom.add_tissue_sphere((-pos,0,0), 32, fat)
test_phantom.add_tissue_sphere((-pos,pos,0), 8, fat)

test_phantom.add_tissue_sphere((0,-pos,0), 32, myocardium)
test_phantom.add_tissue_sphere((0,0,0), 8, myocardium)
test_phantom.add_tissue_sphere((0,pos,0), 16, myocardium)

In [ ]:
plt.imshow(test_phantom.get_complete()[0,:,:,-int(test_phantom.matrix_dims[2] // 2)], vmin=np.amin(test_phantom.get_complete()[0]), vmax=np.amax(test_phantom.get_complete()[0]))
plt.colorbar()

In [ ]:
# # flat calibration phantom

# phantom = Phantom(source_path = None,
#             voxel_dims = (0.1e-3,0.1e-3,0.1e-3),
#             matrix_dims = (400,400,400),
#             baseline = (1500, 1000),
#             seed = 5678,)

# blood = Tissue(name='blood', c=1578, rho=1060, sigma=11.3, scale=0.00001, label=1)
# myocardium = Tissue(name='myocardium', c=1561.3, rho=1081, sigma=30, scale=0.0001, label=2)

# dist = 0.5

# phantom.add_tissue_sphere(( dist, dist, dist),  8, myocardium)
# phantom.add_tissue_sphere(( dist,-dist, dist), 16, myocardium)
# phantom.add_tissue_sphere((-dist,-dist, dist), 32, blood)
# phantom.add_tissue_sphere((-dist, dist, dist), 64, blood)

# phantom.add_tissue_sphere(( dist, dist,-dist), 16, blood)
# phantom.add_tissue_sphere(( dist,-dist,-dist),  8, blood)
# phantom.add_tissue_sphere((-dist,-dist,-dist), 64, myocardium)
# phantom.add_tissue_sphere((-dist, dist,-dist), 32, myocardium)

In [ ]:
# print('Creating phantom...')
# phantom = Phantom.load('hand_phantom')

# phantom.complete.shape
# phantom.complete = phantom.complete[:,200:,:900,200:600]
# phantom.complete.shape

# plt.imshow(phantom.get_complete()[0][:,:,200])
# plt.colorbar()

# Transducer Set

In [ ]:
transducers = [transducer.Focused(elements = 32, 
                       elevation = 1e-4 * 32, 
                       sensor_sampling_scheme = 'not_centroid', 
                       sweep = (np.pi/6,np.pi/6), 
                       ray_num = (32,32), 
                       imaging_ndims = 3,
                       focus_azimuth = 50e-3,
                       focus_elevation = 50e-3,
                       ) for i in range(8)]

# transducers = [transducer.Focused(elements = 64, 
#                        elevation = 1e-4 * 64, 
#                        sensor_sampling_scheme = 'not_centroid', 
#                        sweep = np.pi/4, 
#                        ray_num = 64, 
#                        imaging_ndims = 2,
#                        focus_azimuth = 50e-3,
#                        focus_elevation = 50e-3,
#                        ) for i in range(4)]

for t in transducers:
    t.make_sensor_coords(test_phantom.baseline[0])

In [ ]:
test_transducer_set = transducer_set.TransducerSet(transducers, seed=8888)
test_transducer_set.generate_extrinsics(shape="spherical", extrinsics_kwargs={'r_mean': 50e-3, 'view_std': 0, 'yaw_fraction': 0.3, 'pitch_fraction': 0.6, 'roll_fraction': 0})
test_transducer_set.plot_transducer_fovs(scale=0.05)

In [ ]:
import plotly.graph_objects as go

coords = []
for i in range(len(test_transducer_set.transducers)):
    coords.append(test_transducer_set.poses[i].apply_to_points(test_transducer_set.transducers[i].sensor_coords))
    
fig = go.Figure(data=[go.Scatter3d(
                    x=np.concatenate(coords, axis=0)[:,0],
                    y=np.concatenate(coords, axis=0)[:,1],
                    z=np.concatenate(coords, axis=0)[:,2],
                    mode='markers',
                    marker=dict(size=1, opacity=0.8,),
                )])
fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(scene_aspectmode='data')
fig.show()

# Sensor

In [ ]:
test_sensor = sensor.Sensor(transducer_set=test_transducer_set, aperture_type='single_transducer')

# Simulation

In [ ]:
simprops = simulation.SimProperties(grid_size   = (1024,128,128),
                 PML_size    = (32,8,8),
                 PML_alpha   = 2,
                 t_end       = 12e-5,          # [s]
                 bona        = 6,               # parameter b/a determining degree of nonlinear acoustic effects
                 alpha_coeff = 0.5, 	        # [dB/(MHz^y cm)]
                 alpha_power = 1.5,
                 )

# Experiment

In [ ]:
test_experiment = experiment.Experiment(
                 simulation_path = '../experiment_files/test_experiment_2',
                 sim_properties  = simprops,
                 phantom         = test_phantom,
                 transducer_set  = test_transducer_set,
                 sensor          = test_sensor,
                 nodes           = 2,
                 results         = None,
                 indices         = None,
                 workers         = 3,
                 )

test_experiment.save()

In [ ]:
# test_experiment.save()
test_experiment = experiment.Experiment.load('../experiment_files/test_experiment_2')

In [ ]:
len(test_experiment)

In [ ]:
# test_experiment = Experiment.load('../test_experiment_03')
test_experiment.run(dry=True)

In [ ]:
test_experiment.run()

In [ ]:
test_experiment.add_results()

# Reconstruction

In [ ]:
reconstruction = Reconstruction(test_experiment)

In [ ]:
reconstruction.plot_ray_path(100)

In [ ]:
reconstruction.plot_scatter()